In [28]:
import pandas as pd
import numpy as np
from pprint import pprint
import requests

In [29]:
# I will use this function later to calculate points and wins prior to the race

def lookup (df, team, points):
    df['lookup1'] = df.season.astype(str) + df[team] + df['round'].astype(str)
    df['lookup2'] = df.season.astype(str) + df[team] + (df['round']-1).astype(str)
    new_df = df.merge(df[['lookup1', points]], how = 'left', left_on='lookup2',right_on='lookup1')
    new_df.drop(['lookup1_x', 'lookup2', 'lookup1_y'], axis = 1, inplace = True)
    new_df.rename(columns = {points+'_x': points+'_after_race', points+'_y': points}, inplace = True)
    new_df[points].fillna(0, inplace = True)
    return new_df
   

## Races

In [30]:
races = {'season': [],
        'round': [],
        'circuit_id': [],
        'lat': [],
        'long': [],
        'country': [],
        'date': [],
        'url': []}

for year in list(range(2009,2020)):
    
    url = 'https://ergast.com/api/f1/{}.json'
    r = requests.get(url.format(year))
    json = r.json()

    for item in json['MRData']['RaceTable']['Races']:
        try:
            races['season'].append(int(item['season']))
        except:
            races['season'].append(None)

        try:
            races['round'].append(int(item['round']))
        except:
            races['round'].append(None)

        try:
            races['circuit_id'].append(item['Circuit']['circuitId'])
        except:
            races['circuit_id'].append(None)

        try:
            races['lat'].append(float(item['Circuit']['Location']['lat']))
        except:
            races['lat'].append(None)

        try:
            races['long'].append(float(item['Circuit']['Location']['long']))
        except:
            races['long'].append(None)

        try:
            races['country'].append(item['Circuit']['Location']['country'])
        except:
            races['country'].append(None)

        try:
            races['date'].append(item['date'])
        except:
            races['date'].append(None)

        try:
            races['url'].append(item['url'])
        except:
            races['url'].append(None)
        
races = pd.DataFrame(races)
print(races.shape)

(215, 8)


In [31]:
races.head()

,season,round,circuit_id,lat,long,country,date,url
0,2009,1,albert_park,-37.84970,144.96800,Australia,2009-03-29,http://en.wikipedia.org/wiki/2009_Australian_G...
1,2009,2,sepang,2.76083,101.73800,Malaysia,2009-04-05,http://en.wikipedia.org/wiki/2009_Malaysian_Gr...
2,2009,3,shanghai,31.33890,121.22000,China,2009-04-19,http://en.wikipedia.org/wiki/2009_Chinese_Gran...
3,2009,4,bahrain,26.03250,50.51060,Bahrain,2009-04-26,http://en.wikipedia.org/wiki/2009_Bahrain_Gran...
4,2009,5,catalunya,41.57000,2.26111,Spain,2009-05-10,http://en.wikipedia.org/wiki/2009_Spanish_Gran...


In [32]:
races.tail()

,season,round,circuit_id,lat,long,country,date,url
210,2019,17,suzuka,34.8431,136.5410,Japan,2019-10-13,https://en.wikipedia.org/wiki/2019_Japanese_Gr...
211,2019,18,rodriguez,19.4042,-99.0907,Mexico,2019-10-27,https://en.wikipedia.org/wiki/2019_Mexican_Gra...
212,2019,19,americas,30.1328,-97.6411,USA,2019-11-03,https://en.wikipedia.org/wiki/2019_United_Stat...
213,2019,20,interlagos,-23.7036,-46.6997,Brazil,2019-11-17,https://en.wikipedia.org/wiki/2019_Brazilian_G...
214,2019,21,yas_marina,24.4672,54.6031,UAE,2019-12-01,https://en.wikipedia.org/wiki/2019_Abu_Dhabi_G...


In [33]:
races.to_csv('races.csv', index = False)

## Rounds

In [34]:
race = pd.read_csv('races.csv')

In [35]:
rounds = []
for year in np.array(race.season.unique()):
    rounds.append([year, list(race[race.season == year]['round'])])

In [36]:
rounds[:5]

[[2009, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]],
 [2010, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]],
 [2011, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]],
 [2012,
  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]],
 [2013, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]]]

## Results

In [37]:
results = {'season': [],
          'round':[],
           'circuit_id':[],
          'driver': [],
           'date_of_birth': [],
           'nationality': [],
          'constructor': [],
          'grid': [],
          'time': [],
          'status': [],
          'points': [],
          'podium': [],
          'url': []}

for n in list(range(len(rounds))):
    for i in rounds[n][1]:
    
        url = 'http://ergast.com/api/f1/{}/{}/results.json'
        r = requests.get(url.format(rounds[n][0], i))
        json = r.json()

        for item in json['MRData']['RaceTable']['Races'][0]['Results']:
            try:
                results['season'].append(int(json['MRData']['RaceTable']['Races'][0]['season']))
            except:
                results['season'].append(None)

            try:
                results['round'].append(int(json['MRData']['RaceTable']['Races'][0]['round']))
            except:
                results['round'].append(None)

            try:
                results['circuit_id'].append(json['MRData']['RaceTable']['Races'][0]['Circuit']['circuitId'])
            except:
                results['circuit_id'].append(None)

            try:
                results['driver'].append(item['Driver']['driverId'])
            except:
                results['driver'].append(None)
            
            try:
                results['date_of_birth'].append(item['Driver']['dateOfBirth'])
            except:
                results['date_of_birth'].append(None)
                
            try:
                results['nationality'].append(item['Driver']['nationality'])
            except:
                results['nationality'].append(None)

            try:
                results['constructor'].append(item['Constructor']['constructorId'])
            except:
                results['constructor'].append(None)

            try:
                results['grid'].append(int(item['grid']))
            except:
                results['grid'].append(None)

            try:
                results['time'].append(int(item['Time']['millis']))
            except:
                results['time'].append(None)

            try:
                results['status'].append(item['status'])
            except:
                results['status'].append(None)

            try:
                results['points'].append(int(item['points']))
            except:
                results['points'].append(None)

            try:
                results['podium'].append(int(item['position']))
            except:
                results['podium'].append(None)

            try:
                results['url'].append(json['MRData']['RaceTable']['Races'][0]['url'])
            except:
                results['url'].append(None)

results = pd.DataFrame(results)
print(results.shape)

(4637, 13)


In [38]:
results.head()

,season,round,circuit_id,driver,date_of_birth,nationality,constructor,grid,time,status,points,podium,url
0,2009,1,albert_park,button,1980-01-19,British,brawn,1,5655784.0,Finished,10.0,1,http://en.wikipedia.org/wiki/2009_Australian_G...
1,2009,1,albert_park,barrichello,1972-05-23,Brazilian,brawn,2,5656591.0,Finished,8.0,2,http://en.wikipedia.org/wiki/2009_Australian_G...
2,2009,1,albert_park,trulli,1974-07-13,Italian,toyota,20,5657388.0,Finished,6.0,3,http://en.wikipedia.org/wiki/2009_Australian_G...
3,2009,1,albert_park,glock,1982-03-18,German,toyota,19,5660219.0,Finished,5.0,4,http://en.wikipedia.org/wiki/2009_Australian_G...
4,2009,1,albert_park,alonso,1981-07-29,Spanish,renault,10,5660663.0,Finished,4.0,5,http://en.wikipedia.org/wiki/2009_Australian_G...


In [39]:
results.tail()

,season,round,circuit_id,driver,date_of_birth,nationality,constructor,grid,time,status,points,podium,url
4632,2019,21,yas_marina,giovinazzi,1993-12-14,Italian,alfa,16,NaN,+1 Lap,0.0,16,https://en.wikipedia.org/wiki/2019_Abu_Dhabi_G...
4633,2019,21,yas_marina,russell,1998-02-15,British,williams,18,NaN,+1 Lap,0.0,17,https://en.wikipedia.org/wiki/2019_Abu_Dhabi_G...
4634,2019,21,yas_marina,gasly,1996-02-07,French,toro_rosso,11,NaN,+2 Laps,0.0,18,https://en.wikipedia.org/wiki/2019_Abu_Dhabi_G...
4635,2019,21,yas_marina,kubica,1984-12-07,Polish,williams,19,NaN,+2 Laps,0.0,19,https://en.wikipedia.org/wiki/2019_Abu_Dhabi_G...
4636,2019,21,yas_marina,stroll,1998-10-29,Canadian,racing_point,12,NaN,Brakes,0.0,20,https://en.wikipedia.org/wiki/2019_Abu_Dhabi_G...


In [40]:
results.to_csv('results.csv', index = False)

## Driver Standings

In [41]:
driver_standings = {'season': [],
                    'round':[],
                    'driver': [],
                    'driver_points': [],
                    'driver_wins': [],
                   'driver_standings_pos': []}

for n in list(range(len(rounds))):
    for i in rounds[n][1]:
    
        url = 'https://ergast.com/api/f1/{}/{}/driverStandings.json'
        r = requests.get(url.format(rounds[n][0], i))
        json = r.json()

        for item in json['MRData']['StandingsTable']['StandingsLists'][0]['DriverStandings']:
            try:
                driver_standings['season'].append(int(json['MRData']['StandingsTable']['StandingsLists'][0]['season']))
            except:
                driver_standings['season'].append(None)

            try:
                driver_standings['round'].append(int(json['MRData']['StandingsTable']['StandingsLists'][0]['round']))
            except:
                driver_standings['round'].append(None)
                                         
            try:
                driver_standings['driver'].append(item['Driver']['driverId'])
            except:
                driver_standings['driver'].append(None)
            
            try:
                driver_standings['driver_points'].append(int(item['points']))
            except:
                driver_standings['driver_points'].append(None)
            
            try:
                driver_standings['driver_wins'].append(int(item['wins']))
            except:
                driver_standings['driver_wins'].append(None)
                
            try:
                driver_standings['driver_standings_pos'].append(int(item['position']))
            except:
                driver_standings['driver_standings_pos'].append(None)
            
driver_standings = pd.DataFrame(driver_standings)
print(driver_standings.shape)


(4856, 6)


In [42]:
driver_standings = lookup(driver_standings, 'driver', 'driver_points')

In [43]:
driver_standings = lookup(driver_standings, 'driver', 'driver_wins')

In [44]:
driver_standings = lookup(driver_standings, 'driver', 'driver_standings_pos')

In [45]:
driver_standings.head()

,season,round,driver,driver_points_after_race,driver_wins_after_race,driver_standings_pos_after_race,driver_points,driver_wins,driver_standings_pos
0,2009,1,button,10.0,1,1,0.0,0.0,0.0
1,2009,1,barrichello,8.0,0,2,0.0,0.0,0.0
2,2009,1,trulli,6.0,0,3,0.0,0.0,0.0
3,2009,1,glock,5.0,0,4,0.0,0.0,0.0
4,2009,1,alonso,4.0,0,5,0.0,0.0,0.0


In [46]:
driver_standings.tail()

,season,round,driver,driver_points_after_race,driver_wins_after_race,driver_standings_pos_after_race,driver_points,driver_wins,driver_standings_pos
4851,2019,21,kevin_magnussen,20.0,0,16,20.0,0.0,16.0
4852,2019,21,giovinazzi,14.0,0,17,14.0,0.0,17.0
4853,2019,21,grosjean,8.0,0,18,8.0,0.0,18.0
4854,2019,21,kubica,1.0,0,19,1.0,0.0,19.0
4855,2019,21,russell,0.0,0,20,0.0,0.0,20.0


In [47]:
driver_standings.to_csv('driver_standings.csv', index = False)

## Constructor Standings

In [48]:
constructor_rounds = rounds[8:]

constructor_standings = {'season': [],
                    'round':[],
                    'constructor': [],
                    'constructor_points': [],
                    'constructor_wins': [],
                   'constructor_standings_pos': []}

for n in list(range(len(constructor_rounds))):
    for i in constructor_rounds[n][1]:
    
        url = 'https://ergast.com/api/f1/{}/{}/constructorStandings.json'
        r = requests.get(url.format(constructor_rounds[n][0], i))
        json = r.json()

        for item in json['MRData']['StandingsTable']['StandingsLists'][0]['ConstructorStandings']:
            try:
                constructor_standings['season'].append(int(json['MRData']['StandingsTable']['StandingsLists'][0]['season']))
            except:
                constructor_standings['season'].append(None)

            try:
                constructor_standings['round'].append(int(json['MRData']['StandingsTable']['StandingsLists'][0]['round']))
            except:
                constructor_standings['round'].append(None)
                                         
            try:
                constructor_standings['constructor'].append(item['Constructor']['constructorId'])
            except:
                constructor_standings['constructor'].append(None)
            
            try:
                constructor_standings['constructor_points'].append(int(item['points']))
            except:
                constructor_standings['constructor_points'].append(None)
            
            try:
                constructor_standings['constructor_wins'].append(int(item['wins']))
            except:
                constructor_standings['constructor_wins'].append(None)
                
            try:
                constructor_standings['constructor_standings_pos'].append(int(item['position']))
            except:
                constructor_standings['constructor_standings_pos'].append(None)
            
constructor_standings = pd.DataFrame(constructor_standings)
print(constructor_standings.shape)


(620, 6)


In [49]:
constructor_standings = lookup(constructor_standings, 'constructor', 'constructor_points')

In [50]:
constructor_standings = lookup(constructor_standings, 'constructor', 'constructor_wins')

In [51]:
constructor_standings = lookup(constructor_standings, 'constructor', 'constructor_standings_pos')

In [52]:
constructor_standings.head()

,season,round,constructor,constructor_points_after_race,constructor_wins_after_race,constructor_standings_pos_after_race,constructor_points,constructor_wins,constructor_standings_pos
0,2017,1,ferrari,37,1,1,0.0,0.0,0.0
1,2017,1,mercedes,33,0,2,0.0,0.0,0.0
2,2017,1,red_bull,10,0,3,0.0,0.0,0.0
3,2017,1,williams,8,0,4,0.0,0.0,0.0
4,2017,1,force_india,7,0,5,0.0,0.0,0.0


In [53]:
constructor_standings.tail()

,season,round,constructor,constructor_points_after_race,constructor_wins_after_race,constructor_standings_pos_after_race,constructor_points,constructor_wins,constructor_standings_pos
615,2019,21,toro_rosso,85,0,6,83.0,0.0,6.0
616,2019,21,racing_point,73,0,7,67.0,0.0,7.0
617,2019,21,alfa,57,0,8,57.0,0.0,8.0
618,2019,21,haas,28,0,9,28.0,0.0,9.0
619,2019,21,williams,1,0,10,1.0,0.0,10.0


In [54]:
constructor_standings.to_csv('constructor_standings.csv', index = False)